In [ ]:
import numpy as np
import pandas as pd
import os
import h5py
import matplotlib
from scipy import signal
from matplotlib import pyplot as plt
# %matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Keras
import keras
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras import backend as K
from keras import regularizers
from tqdm import tqdm
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
# Custom imports
import scipy.io


In [ ]:
import pickle
with open("../input/dataLabel.p", 'rb') as f:
  label_dict, enc_dict = pickle.load(f)

In [ ]:
import pickle
with open("../input/fullData.p", 'rb') as f:
  lengths, ecgData, ecgKeys = pickle.load(f)

In [ ]:
import numpy as np
from scipy import signal
import scipy as sc

def extend_ts(ts, length):
    extended = np.zeros(length)
    siglength = np.min([length, ts.shape[0]])
    extended[:siglength] = ts[:siglength]
    return extended 

def zero_filter(input, threshold=2, depth=8):
    shape = input.shape
    # compensate for lost length due to mask processing
    noise_shape = [shape[0], shape[1] + depth]
    
    # Generate random noise
    noise = np.random.normal(0,1,noise_shape)
    
    # Pick positions where the noise is above a certain threshold
    mask = np.greater(noise, threshold)
    
    # grow a neighbourhood of True values with at least length depth+1
    for d in range(depth):
        mask = np.logical_or(mask[:, :-1], mask[:, 1:])
    output = np.where(mask, np.zeros(shape), input)
    return output

def stretch_squeeze(source, length):
    target = np.zeros([1, length])
    interpol_obj = sc.interpolate.interp1d(np.arange(source.size), source)
    grid = np.linspace(0, source.size - 1, target.size)
    result = interpol_obj(grid)
    return result

def fit_tolength(source, length):
    target = np.zeros([length])
    w_l = min(source.size, target.size)
    target[0:w_l] = source[0:w_l]
    return target

# Data augmentation scheme: Random resampling
def random_resample(signals, upscale_factor = 1):
    [n_signals,length] = signals.shape
    # pulse variation from 60 bpm to 120 bpm, expected 80 bpm
    new_length = np.random.randint(
        low=int(length*80/120),
        high=int(length*80/60),
        size=[n_signals, upscale_factor])
    signals = [np.array(s) for s in signals.tolist()]
    new_length = [np.array(nl) for nl in new_length.tolist()]
    sigs = [stretch_squeeze(s,l) for s,nl in zip(signals,new_length) for l in nl]
    sigs = [fit_tolength(s, length) for s in sigs]
    sigs = np.array(sigs)
    return sigs

def spectrogram(data, nperseg=64, noverlap=32, log_spectrogram = True, fs=300):
    f, t, Sxx = signal.spectrogram(data, fs=fs, nperseg=nperseg, noverlap=noverlap)
#     print(Sxx.shape)
    Sxx = np.transpose(Sxx,[0,2,1])
    if log_spectrogram:
        Sxx = abs(Sxx) # Make sure, all values are positive before taking log
        mask = Sxx > 0 # We dont want to take the log of zero
        Sxx[mask] = np.log(Sxx[mask])
    return f, t, Sxx

# Spectrogram statistics needed for normalization of the data set
def transformed_stats(ecgData, ecgKeys, nperseg, noverlap, sequence_length):

    ''' Gets some important statistics of the spectrograms in the entire dataset.
    We need this to rescale the data'''

    dataset_list = ecgKeys
    sample_list = []

    for dataset in dataset_list:
        data = extend_ts(ecgData[dataset], sequence_length)
        data = np.reshape(data, (1, len(data)))
        sample_list.append(np.expand_dims(spectrogram(data, nperseg, noverlap)[2], axis = 3))
    
    sample_array = np.vstack(sample_list)
    
    #Flatten the array so that we can do statistics
    samples = np.ndarray.flatten(sample_array)
        
    return np.min(samples), np.max(samples), np.mean(samples), np.std(samples)

# Float types are normalized to zero mean std 
def norm_float(data, data_mean, data_std):
    scaled = (data - data_mean)/data_std
    return scaled



In [ ]:
import numpy as np
import keras


def generator(ecgData, ecgKeys, label_dict, batch_size = 32, dim = (178, 33), 
                 nperseg = 64, noverlap = 32, data_mean = -9.01, data_std = 9.00,  
                 n_channels=1, sequence_length = 5736, 
                 n_classes = 4, shuffle = True, augment = False):
    while True:
        X = np.empty((batch_size, dim[0], dim[1], n_channels), dtype = float)
        y = np.empty((batch_size), dtype = int)

        randomIds = np.random.randint(0, len(ecgKeys), batch_size)

        for i, ID in enumerate(randomIds):
            data = extend_ts(ecgData[ecgKeys[ID]], sequence_length)
            data = np.reshape(data, (1, len(data)))

            if augment:

                # dropout bursts
                data = zero_filter(data, threshold = 2, depth = 10)

                # random resampling
                data = random_resample(data)

            # Generate spectrogram
            data_spectrogram = spectrogram(data, nperseg = nperseg, noverlap = noverlap)[2]

            # Normalize
            data_transformed = norm_float(data_spectrogram, data_mean, data_std)

            X[i,] = np.expand_dims(data_transformed, axis = 3)

            # Assuming that the dataset names are unique (only 1 per label)
            y[i] = label_dict[ecgKeys[ID]]

        yield X, keras.utils.to_categorical(y, num_classes=n_classes)

In [ ]:
# Spectrogram Parameters
Fs = 300
NFFT = 64
NOVERLAP = 32
N_classes = 4
BATCH_SIZE = 32
MAX_LENGTH = 9000

In [ ]:
_, _, Sxx = spectrogram(ecgData[ecgKeys[0]].reshape(1, 9000), nperseg = NFFT, noverlap = NOVERLAP)
DIM = Sxx[0].shape

In [ ]:
MIN, MAX, MEAN, STD = transformed_stats(ecgData, ecgKeys, NFFT, NOVERLAP, MAX_LENGTH)

In [ ]:
gen = generator(ecgData, ecgKeys, label_dict, augment=True,batch_size=32, dim=DIM, nperseg=NFFT, noverlap=NOVERLAP, data_mean=MEAN, data_std=STD, n_channels=1, sequence_length=9000, n_classes=4, shuffle=True)

In [ ]:
# Convolutional blocks
def conv2d_block(model, depth, layer_filters, filters_growth, 
                 strides_start, strides_end, input_shape, first_layer = False):
    
    ''' Convolutional block. 
    depth: number of convolutional layers in the block (4)
    filters: 2D kernel size (32)
    filters_growth: kernel size increase at the end of block (32)
    first_layer: provide input_shape for first layer'''
    
    # Fixed parameters for convolution
    conv_parms = {'kernel_size': (3, 3),
                  'padding': 'same',
                  'dilation_rate': (1, 1),
                  'activation': None,
                  'data_format': 'channels_last',
                  'kernel_initializer': 'glorot_normal'}

    for l in range(depth):

        if first_layer:
            
            # First layer needs an input_shape 
            model.add(layers.Conv2D(filters = layer_filters,
                                    strides = strides_start,
                                    input_shape = input_shape, **conv_parms))
            first_layer = False
        
        else:
            # All other layers will not need an input_shape parameter
            if l == depth - 1:
                # Last layer in each block is different: adding filters and using stride 2
                layer_filters += filters_growth
                model.add(layers.Conv2D(filters = layer_filters,
                                        strides = strides_end, **conv_parms))
            else:
                model.add(layers.Conv2D(filters = layer_filters,
                                        strides = strides_start, **conv_parms))
        
        # Continue with batch normalization and activation for all layers in the block
        model.add(layers.BatchNormalization(center = True, scale = True))
        model.add(layers.Activation('relu'))
    
    return model

def MeanOverTime():
    lam_layer = layers.Lambda(lambda x: K.mean(x, axis=1), output_shape=lambda s: (1, s[2]))
    return lam_layer

In [ ]:
# Define the model
# Model parameters
filters_start = 32 # Number of convolutional filters
layer_filters = filters_start # Start with these filters
filters_growth = 32 # Filter increase after each convBlock
strides_start = (1, 1) # Strides at the beginning of each convBlock
strides_end = (2, 2) # Strides at the end of each convBlock
depth = 4 # Number of convolutional layers in each convBlock
n_blocks = 6 # Number of ConBlocks
n_channels = 1 # Number of color channgels
input_shape = (280, 33, 1) # input shape for first layer


model = Sequential()

for block in range(n_blocks):

    # Provide input only for the first layer
    if block == 0:
        provide_input = True
    else:
        provide_input = False
    
    model = conv2d_block(model, depth,
                         layer_filters,
                         filters_growth,
                         strides_start, strides_end,
                         input_shape,
                         first_layer = provide_input)
    
    # Increase the number of filters after each block
    layer_filters += filters_growth



# Remove the frequency dimension, so that the output can feed into LSTM
# Reshape to (batch, time steps, filters)
model.add(layers.Reshape((-1, 224)))
model.add(layers.core.Masking(mask_value = 0.0))
model.add(MeanOverTime())

# Alternative: Replace averaging by LSTM

# Insert masking layer to ignore zeros
#model.add(layers.core.Masking(mask_value = 0.0))

# Add LSTM layer with 3 neurons
#model.add(layers.LSTM(200))
#model.add(layers.Flatten())

# And a fully connected layer for the output
model.add(layers.Dense(4, activation='sigmoid', kernel_regularizer = regularizers.l2(0.1)))


model.summary()

In [ ]:
model = keras.models.load_model("../input/modelCNN (1).h5")

In [ ]:
# Compile the model and run a batch through the network
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.000001, decay=0.09),
              metrics=['acc'])

model_checkpoint = ModelCheckpoint("modelCNN.h5", save_best_only=False, save_weights_only=False)

In [ ]:
model.fit_generator(gen,steps_per_epoch = 800,epochs = 100, callbacks=[model_checkpoint])